In [162]:
import ccxt 
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
FTX_API_KEY = os.getenv("FTX_API_KEY")
FTX_API_SECRET = os.getenv("FTX_API_SECRET")
SUBACCOUNT = 'FT4'

ftx = ccxt.ftx(
    {
        'apiKey': FTX_API_KEY,
        'secret': FTX_API_SECRET,
        'enableRateLimit': True,
        'headers': {
            'FTX-SUBACCOUNT': SUBACCOUNT,
        },
    }
)


In [163]:
data = ftx.fetch_my_trades()
trades = pd.DataFrame(data)

In [164]:
trades_cleaned = pd.json_normalize(trades['info'])
trades = pd.concat([trades.drop(['info'], axis=1), trades_cleaned], axis=1)
trades['price'] = trades['price'].astype(float)
trades['size'] = trades['size'].astype(float)
trades_cleaned = trades_cleaned.drop(['id', 'fee', 'feeCurrency'], axis=1)
display(trades_cleaned)

,market,future,baseCurrency,quoteCurrency,type,side,price,size,orderId,time,tradeId,feeRate,liquidity
0,ETHW-PERP,ETHW-PERP,None,None,order,sell,6.066,25.0,191994647306,2022-10-20T20:32:56.831048+00:00,5186143039,-5e-6,maker
1,ETHW-PERP,ETHW-PERP,None,None,order,sell,6.118,25.0,191996015234,2022-10-20T20:37:33.976391+00:00,5186177539,-5e-6,maker
2,ETHW-PERP,ETHW-PERP,None,None,order,sell,6.147,25.0,191998088235,2022-10-20T20:46:53.131614+00:00,5186219132,-5e-6,maker
3,ETHW/USD,None,ETHW,USD,order,buy,6.146,25.0,191999335469,2022-10-20T20:53:20.630352+00:00,5186240603,-5e-6,maker
4,ETHW-PERP,ETHW-PERP,None,None,order,buy,6.146,25.0,192000162510,2022-10-20T20:57:35.094696+00:00,5186255289,-5e-6,maker
5,ETHW-PERP,ETHW-PERP,None,None,order,buy,6.149,25.0,192000223578,2022-10-20T20:57:47.942177+00:00,5186256068,-5e-6,maker
6,ETHW-PERP,ETHW-PERP,None,None,order,sell,6.145,0.1,192000370675,2022-10-20T20:58:25.879391+00:00,5186259317,-5e-6,maker
7,ETHW-PERP,ETHW-PERP,None,None,order,sell,6.145,24.9,192000370675,2022-10-20T20:58:34.963779+00:00,5186259725,-5e-6,maker
8,ETHW/USD,None,ETHW,USD,order,buy,6.147,25.0,192000532500,2022-10-20T20:59:30.114343+00:00,5186261934,-5e-6,maker
9,ETHW-PERP,ETHW-PERP,None,None,order,sell,6.145,25.0,192000587340,2022-10-20T20:59:47.100234+00:00,5186262781,-5e-6,maker


In [165]:
df = trades_cleaned

start = pd.to_datetime(df['time'].min())
end = pd.to_datetime(df['time'].max())
delta = end - start

spot = df.loc[df['future'].isnull()]
perp = df.loc[df['future'].notnull()]

spot['price'] = spot['price'].astype(float)
spot['size'] = spot['size'].astype(float)
perp['price'] = perp['price'].astype(float)
perp['size'] = perp['size'].astype(float)
perp['side']
perp.loc[perp['side'] == 'buy', 'size'] = -perp['size']
spot.loc[spot['side'] == 'sell', 'size'] = -spot['size']

# get the start and end time delta in the df 
start = pd.to_datetime(df['time'].min())
end = pd.to_datetime(df['time'].max())
delta = end - start

spot_avg_entry = np.average(spot['price'], weights=spot['size'])
perp_avg_entry = np.average(perp['price'], weights=perp['size'])
entry_spread = round((perp_avg_entry - spot_avg_entry)*100, 6)
spread_entry_gain_loss_percent = round((entry_spread / perp_avg_entry), 6) 

data = {
    'spot_avg_entry': spot_avg_entry, 
    'perp_avg_entry': perp_avg_entry,
    'entry_spread': entry_spread,
    'spread_entry_gain_loss_percent': spread_entry_gain_loss_percent,
    'spot_market': spot['market'].unique()[0],
    'perp_market': perp['market'].unique()[0],
    'spot_size': spot['size'].sum(),
    'perp_size': perp['size'].sum(),
    'start': start,
    'end': end,
    'entry_total_time': delta,
}

In [166]:
print(data)

{'spot_avg_entry': 6.148000000000001, 'perp_avg_entry': 6.108666666666666, 'entry_spread': -3.933333, 'spread_entry_gain_loss_percent': -0.643894, 'spot_market': 'ETHW/USD', 'perp_market': 'ETHW-PERP', 'spot_size': 75.0, 'perp_size': 75.0, 'start': Timestamp('2022-10-20 20:32:56.831048+0000', tz='UTC'), 'end': Timestamp('2022-10-20 21:01:10.281226+0000', tz='UTC'), 'entry_total_time': Timedelta('0 days 00:28:13.450178')}
